In [ ]:
%matplotlib widget

In [ ]:
import numpy as np
from lotr import plotting as pltltr
from matplotlib import pyplot as plt
from tifffile import imread

COLS = pltltr.COLS

In [ ]:
stack = imread("/Volumes/Shared/confocal/dIPN_quarter.tif")

In [ ]:
stack.shape

In [ ]:
plt.figure()
plt.imshow(stack[100, 0, :, :])

In [ ]:
ph_cols = COLS["phase"](np.arange(-np.pi, np.pi, np.pi / 2))

In [ ]:
import colorspacious


def JCh_to_RGB255(x):
    output = np.clip(colorspacious.cspace_convert(x, "JCh", "sRGB1"), 0, 1)
    return (output * 255).astype(np.uint8)


def color_img(
    amp,
    angle,
    hueshift=0,
    amp_percentile=80,
    maxsat=50,
    lightness_min=100,
    lightness_delta=-40,
):
    output_lch = np.empty(amp.shape + (3,))
    maxamp = np.percentile(amp, amp_percentile)

    output_lch[:, :, 0] = (
        lightness_min + (np.clip(amp / maxamp, 0, 1)) * lightness_delta
    )
    output_lch[:, :, 1] = (np.clip(amp / maxamp, 0, 1)) * maxsat
    output_lch[:, :, 2] = (angle + hueshift) * 180 / np.pi

    return JCh_to_RGB255(output_lch)

In [ ]:
plt.close("all")

In [ ]:
img = stack[p[0] : p[1], i, y_crop[0] : y_crop[1], x_crop[0] : x_crop[1]].mean(0)
c_img = color_img(img, np.full(img.shape, 0), amp_percentile=99.0, lightness_delta=-40)
plt.figure()
plt.imshow(c_img)

In [ ]:
f, axs = plt.subplots(5, 2, figsize=(6, 10))
x_crop = (300, 500)
y_crop = (500, 600)
planes = [(160, 180), (100, 130)]

all_c_ipn = []
all_c_ahb = []
for i in range(4):
    for j, p in enumerate(planes):
        img = stack[p[0] : p[1], i, y_crop[0] : y_crop[1], x_crop[0] : x_crop[1]].mean(
            0
        )
        c_img = color_img(
            img,
            np.full(img.shape, -np.pi + np.pi / 2 * i),
            amp_percentile=99.5,
            lightness_delta=-80,
        )
        [all_c_ipn, all_c_ahb][j].append(img)
        axs[i, j].imshow(c_img, origin="upper")

all_c_ipn = np.array(all_c_ipn)
all_c_ahb = np.array(all_c_ahb)

In [ ]:
def get_normalized_colored(stack):
    norm = np.array(
        [stack[c, :, :] - np.percentile(stack[c, :, :], 1) for c in range(4)]
    )
    amp = norm.max((0))
    angle = np.arctan2(
        np.mean([norm[0, :, :], -norm[2, :, :]], axis=0),
        np.mean([norm[1, :, :], -norm[3, :, :]], axis=0),
    )
    return color_img(amp, angle, amp_percentile=99.0, lightness_delta=-80,)

In [ ]:
f, axs = plt.subplots(1, 2, figsize=(6, 2))
axs[0].imshow(get_normalized_colored(all_c_ipn), origin="upper")
axs[1].imshow(get_normalized_colored(all_c_ahb), origin="upper")

In [ ]:
sum_ipn = ((all_c_ipn * 0.25).sum(0)).astype(int)
sum_ipn[sum_ipn > 255] = 255

plt.figure()
plt.imshow(sum_ipn)